In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
cd /content/drive/MyDrive/IMDB

/content/drive/MyDrive/IMDB


In [71]:
# !sudo pip3 install dgl-cu101
#102, 117, 118 tried doesn't work, 101 doesn't exist
# !pip uninstall dgl -y
# !pip install dgl -f https://data.dgl.ai/wheels/cu117/repo.html
# # !git clone  https://github.com/awslabs/dgl-ke.git
# # !pip uninstall dglke -y
# # !sudo pip3 install dglke
# # !pushd dgl-ke;cd python;sudo python3 setup.py install;
# !pip3 uninstall torch -y
# # !sudo pip3 install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


In [72]:
pip install -U sentence-transformers

In [73]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git


Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 39.9 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 21.1 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.2 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-rumrypen
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-rumrypen
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit e0a2b2aaa621d811cb2115c15766d7235aae0b87
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.4.0-py3-none-any.whl size=10

In [74]:
import pandas as pd
import torch
from torch_geometric.data import HeteroData

data = pd.read_csv('IMDB-Movie-Data.csv')

counter = 0
node_id_mapping = {'title':0,'genre':1,'actor':2,'director':3,'year':4}
id_node_mapping = {}
heteroData = HeteroData()
# metapath = [
#       ("title", "is_of", "genre"),
#       ("actor", "acts_in", "title"),
#       ("director", "directs", "title"),
#       ("title", "is_when", "year")
#   ]
metapath = [
    ("title", "is_of", "genre"),
    ("genre", "type_of" ,"title"),
    ("title", "features", "actor"),
    ("actor", "acts_in", "title"),
    ("title", "is_directed_by", "director"),
    ("director", "directs", "title"),
    ("title", "is_when", "year"),
    ("year","is_the_timeline","title")
]
for met in metapath:
  heteroData[met].edge_index = [[], []]
  # heteroData[met] = [[], []]
for i in data.index:
  title = data['Title'][i] + '___' + str(data['Year'][i])
  genre = data['Genre'][i].split(',')
  actors = data['Actors'][i].split(',')
  director = data['Director'][i]
  year = str(data['Year'][i])
  node_id_mapping[title]= ['title',counter]
  id_node_mapping[counter] = ['title',title]
  title_id = counter
  counter+=1

  for g in genre:
    if g not in node_id_mapping.keys():
      node_id_mapping[g]= ['genre',counter]
      id_node_mapping[counter] = ['genre',g]
      met = ("title", "is_of", "genre")
      heteroData[met].edge_index[0].append(title_id)
      heteroData[met].edge_index[1].append(counter)
      # heteroData[met].append(title_id)
      # heteroData[met].append(counter)
      met = ("genre", "type_of" ,"title")
      heteroData[met].edge_index[0].append(counter)
      heteroData[met].edge_index[1].append(title_id)
      # heteroData[met].append(counter)
      # heteroData[met].append(title_id)
      counter+=1
    else:
      met = ("title", "is_of", "genre")
      heteroData[met].edge_index[0].append(title_id)
      heteroData[met].edge_index[1].append(node_id_mapping[g][1])
      # heteroData[met].append(title_id)
      # heteroData[met].append(node_id_mapping[g])
      met = ("genre", "type_of" ,"title")
      heteroData[met].edge_index[0].append(node_id_mapping[g][1])
      heteroData[met].edge_index[1].append(title_id)
      # heteroData[met].append(node_id_mapping[g])
      # heteroData[met].append(title_id)

  for a in actors:
    if a not in node_id_mapping.keys():
      node_id_mapping[a]= ['actor',counter]
      id_node_mapping[counter] = ['actor',a]
      met = ("actor", "acts_in", "title")
      heteroData[met].edge_index[0].append(counter)
      heteroData[met].edge_index[1].append( title_id)
      # heteroData[met].append(counter)
      # heteroData[met].append( title_id)
      met = ("title", "features", "actor")
      heteroData[met].edge_index[0].append(title_id)
      heteroData[met].edge_index[1].append(counter)
      # heteroData[met].append(title_id)
      # heteroData[met].append(counter)
      counter+=1
    else:
      met = ("actor", "acts_in", "title")
      heteroData[met].edge_index[0].append(node_id_mapping[a][1])
      heteroData[met].edge_index[1].append( title_id)
      # heteroData[met].append(node_id_mapping[a])
      # heteroData[met].append( title_id)
      met = ("title", "features", "actor")
      heteroData[met].edge_index[0].append(title_id)
      heteroData[met].edge_index[1].append(node_id_mapping[a][1])
      # heteroData[met].append(title_id)
      # heteroData[met].append(node_id_mapping[a])

  if director not in node_id_mapping.keys():
    node_id_mapping[director]= ['director',counter]
    id_node_mapping[counter] = ['director',director]
    met = ("director", "directs", "title")
    heteroData[met].edge_index[0].append(counter)
    heteroData[met].edge_index[1].append(title_id)
    # heteroData[met].append(counter)
    # heteroData[met].append(title_id)
    met = ("title", "is_directed_by", "director")
    heteroData[met].edge_index[0].append(title_id)
    heteroData[met].edge_index[1].append(counter)
    # heteroData[met].append(title_id)
    # heteroData[met].append(counter)
    counter+=1
  else:
    met = ("director", "directs", "title")
    heteroData[met].edge_index[0].append(node_id_mapping[director][1])
    heteroData[met].edge_index[1].append(title_id)
    # heteroData[met].append(node_id_mapping[director])
    # heteroData[met].append(title_id)
    met = ("title", "is_directed_by", "director")
    heteroData[met].edge_index[0].append(title_id)
    heteroData[met].edge_index[1].append(node_id_mapping[director][1])
    # heteroData[met].append(title_id)
    # heteroData[met].append(node_id_mapping[director])

  if year not in node_id_mapping.keys():
    node_id_mapping[year]= ['year',counter]
    id_node_mapping[counter] = ['year',year]
    met = ("title", "is_when", "year")
    heteroData[met].edge_index[0].append(title_id)
    heteroData[met].edge_index[1].append(counter)
    # heteroData[met].append(title_id)
    # heteroData[met].append(counter)
    met = ("year","is_the_timeline","title")
    heteroData[met].edge_index[0].append(counter)
    heteroData[met].edge_index[1].append(title_id)
    # heteroData[met].append(counter)
    # heteroData[met].append(title_id)
    counter+=1
  else:
    met = ("title", "is_when", "year")
    heteroData[met].edge_index[0].append(title_id)
    heteroData[met].edge_index[1].append(node_id_mapping[year][1])
    # heteroData[met].append(title_id)
    # heteroData[met].append(node_id_mapping[year])
    met = ("year","is_the_timeline","title")
    heteroData[met].edge_index[0].append(node_id_mapping[year][1])
    heteroData[met].edge_index[1].append(title_id)
    # heteroData[met].append(node_id_mapping[year])
    # heteroData[met].append(title_id)



In [75]:
import torch_geometric.transforms as T

for met in metapath:
  # heteroData[met] = torch.tensor(heteroData[met], dtype = torch.long)
  heteroData[met].edge_index = torch.tensor(heteroData[met].edge_index, dtype = torch.long)

heteroData.edge_index_dict
# print(heteroData.edge_index_dict)
# transform = T.RandomLinkSplit(is_undirected=True,num_val = 0.1,
#               num_test = 0.1,
#               edge_types = [("title", "is_when", "year")],
#               rev_edge_types = [("year","is_the_timeline","title")])
# # # print(heteroData.metadata())
# # edge_store = heteroData.get_edge_store("title", "is_when", "year")
# # edge_index = edge_store['edge_index']
# # row, col = edge_index
# # print(row*2)
# edge_types = [("title", "is_when", "year")]
# stores = [heteroData[edge_type] for edge_type in edge_types]
# print(stores[0])
# for item in stores:
#   store = item
#   print(store.size())
# train_data, val_data, test_data = transform(heteroData)



{('title',
  'is_of',
  'genre'): tensor([[   0,    0,    0,  ..., 4044, 4044, 4044],
         [   1,    2,    3,  ...,   29,   30,   37]]),
 ('genre',
  'type_of',
  'title'): tensor([[   1,    2,    3,  ...,   29,   30,   37],
         [   0,    0,    0,  ..., 4044, 4044, 4044]]),
 ('title',
  'features',
  'actor'): tensor([[   0,    0,    0,  ..., 4044, 4044, 4044],
         [   4,    5,    6,  ...,  413, 2894, 4046]]),
 ('actor',
  'acts_in',
  'title'): tensor([[   4,    5,    6,  ...,  413, 2894, 4046],
         [   0,    0,    0,  ..., 4044, 4044, 4044]]),
 ('title',
  'is_directed_by',
  'director'): tensor([[   0,   10,   18,  ..., 4034, 4039, 4044],
         [   8,   16,   25,  ...,  880, 4043, 2225]]),
 ('director',
  'directs',
  'title'): tensor([[   8,   16,   25,  ...,  880, 4043, 2225],
         [   0,   10,   18,  ..., 4034, 4039, 4044]]),
 ('title',
  'is_when',
  'year'): tensor([[   0,   10,   18,  ..., 4034, 4039, 4044],
         [   9,   17,   26,  ...,  341,    

In [76]:
from torch_geometric.nn import MetaPath2Vec
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# for keys, edge_index in heteroData.edge_index_dict.items():
#     key = keys[0]
#     try:
#       N = int(edge_index[0].max() + 1)
#       print(N)
#     except:
#       print(edge_index[0])
model = MetaPath2Vec(
    heteroData.edge_index_dict,
    embedding_dim=384,
    metapath=metapath, walk_length=50, context_size=7,
    walks_per_node=5, num_negative_samples=5,
    sparse=True).to(device)


# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer = torch.optim.SparseAdam(model.parameters(), lr=0.01)

In [77]:
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [78]:
pos_rw, neg_rw = next(iter(loader))
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)



In [79]:
# epochs = 0
# prev_loss = -1
# loss = 0
# for i in range(15):
#   prev_loss = loss
#   loss = train()
#   epochs +=1
#   print(f'epochs - {epochs}')
#   print(loss)


In [80]:
# import pickle as pkl

# with open('model.pkl','wb') as file:
#   pkl.dump(model,file)

In [81]:
import pickle as pkl
with open('model.pkl','rb') as file:
  model = pkl.load(file)

In [82]:
def get_embedding_from_nodetxt(model,text):
  typ,id = node_id_mapping[text]
  # print(typ)
  input = torch.tensor([id])
  embedding = model(typ,input)
  return embedding



In [83]:
from torch.nn import CosineSimilarity
def get_similar_nodeemb_from_emb(model,emb):
  nodeid=-1
  nodetxt=''
  nodeemb = []
  maxcos=-2
  cossim = CosineSimilarity()
  for id,val in id_node_mapping.items():
    # print(val)
    temp = model(val[0],torch.tensor([id],device=device))
    emb = emb.cuda(temp.get_device()) if temp.is_cuda else emb

    simval = cossim(emb,temp)

    if simval>maxcos:
      nodeid = id
      nodetxt = val[1]
      nodeemb = temp
      maxcos = simval
  return nodeid, nodetxt, nodeemb

# print(get_similar_nodeemb_from_emb(model,model('title',torch.tensor([0],device=device))))

In [84]:
from sentence_transformers import SentenceTransformer
st_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
for param in st_model.parameters():
    param.requires_grad = False


In [85]:
qanda = []
alltext = ""
with open("dataset.txt", "r") as file:
  for line in file:
    alltext += line

qanda = alltext.split("\n\n")

qanda_sep = []
for qa in qanda:
  qa = qa.split("\n")
  if(len(qa)<2):
    continue
  q = qa[0]
  a = qa[1][8:]
  qanda_sep.append([q,a])

In [86]:
print(qanda_sep)

[['In which year was the movie "Interstellar" with Matthew McConaughey released?', '2014'], ['Identify a 2015 film starring Jennifer Lawrence.', 'The Hunger Games: Mockingjay - Part 2'], ['Name a movie released in 2013 with Christian Bale in the lead role.', 'American Hustle'], ['Can you mention a 2016 release that shares its lead actor with "Deadpool"?', 'Deadpool'], ['Which film stars Leonardo DiCaprio and was released in 2015?', 'The Revenant'], ['What is the title of the 2014 movie featuring Scarlett Johansson?', 'Lucy'], ['Which 2016 movie stars Henry Cavill and Ben Affleck?', 'Batman v Superman: Dawn of Justice'], ['In which year was the movie "The Martian" with Matt Damon released?', '2015'], ['Identify a film from 2013 that stars Sandra Bullock.', 'Gravity'], ['Name a 2013 release with Hugh Jackman as the lead actor.', 'The Wolverine'], ['Can you mention a 2012 movie with Anne Hathaway in the lead role?', 'Les Misérables'], ['What is the title of the film from 2016 featuring Ma

In [87]:
import torch
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Assuming your DataFrame has columns 'feature_column' and 'label_column'
        text = self.data[idx][0]
        node_id = self.data[idx][1]
        # features = features.to(device)
        # label = label.to(device)
        return text, node_id

In [88]:
# from torch.utils.data import DataLoader

# dataset = CustomDataset(qanda_sep)

# train_dataloader = DataLoader(dataset, shuffle=True, batch_size=8)


In [89]:
# from sentence_transformers import SentenceTransformer, InputExample, losses
# from torch.utils.data import DataLoader
# train_loss = losses.CosineSimilarityLoss(st_model)
# training_examples = [InputExample(texts = i,label=1.0) for i in qanda_sep]
# # training_examples.extend([InputExample(texts = i,label=0.0) for i in neg_samples])
# train_dataloader = DataLoader(training_examples, shuffle=True, batch_size=8)
# st_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, warmup_steps=100)

In [90]:
import pandas as pd
df = pd.read_csv('IMDB-Movie-Data.csv')

In [91]:
from torch.optim import AdamW
from torch.nn import CosineEmbeddingLoss
from torch.nn import Linear
from torch import from_numpy
device = 'cuda' if torch.cuda.is_available() else 'cpu'

optimizer = AdamW(st_model.parameters(), lr=1e-3)
from transformers import get_scheduler
cosine_loss = torch.nn.CosineEmbeddingLoss()

new_layer = Linear(384,384, device=device)

num_epochs = 3
num_training_steps = num_epochs * 160
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# for epoch in range(1):
#     for batch in train_dataloader:
#         inp , val = batch
#         # print(val)
#         # batch = {k: v.to(device) for k, v in batch.items()}
#         # print(batch)
#         emb = st_model(batch)
#         # qemb = st_model.encode(inp)
#         # aemb = st_model.encode(val)
#         # print(len(outputs[0]))
#         targets = torch.ones(len(batch[0]))
#         # print(targets)]
#         ans_emb=[]
#         # for a in batch[1]:
#         #   try:
#         #     model.wv[a]
#         #   except:
#         #     for
#         #       model.wv[]
#         # loss = cosine_loss(model.wv[], emb,targets )
#         # # print(loss)
#         # loss.backward()

#         # optimizer.step()
#         # lr_scheduler.step()
#         # optimizer.zero_grad()

for epoch in range(5):
  for q,a in qanda_sep:
    try:
      qemb = st_model.encode(a)
      qemb = torch.tensor(qemb,device=device)
      qemb = new_layer(qemb)
      targets = torch.ones(1, device = device)
      aemb = []
      try:
        aemb = get_embedding_from_nodetxt(model,a)
      except:
        row = df.loc[df['Title'] == a]
        temp = a+'___'+str(df['Year'][row.index[0]])
        aemb = get_embedding_from_nodetxt(model,a)
      aemb = torch.tensor(aemb, device=device)
      aemb = torch.reshape(aemb,(1,384))
      qemb = torch.unsqueeze(qemb, 0)
      loss =  cosine_loss(aemb, qemb,targets )
      print(loss)
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
    except:
      continue


In [97]:
from torch import from_numpy
for q,a in qanda_sep:
  emb = st_model.encode(q)
  emb = from_numpy(emb)
  emb = torch.tensor(emb,device=device)
  emb = new_layer(emb)
  emb = emb.reshape((1,384))
  # print(emb.shape)
  # print(emb)
  nodeid, nodetxt, nodeemb = get_similar_nodeemb_from_emb(model,emb)
  print(f'actual = {a}',f'retrieved = {nodetxt}')
  # print(nodetxt)



<ipython-input-97-bb02c9da03e5>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb,device=device)
